## Setup Equilibrium object 
setting up eq object to use in testing the functionality of the Probe class.

This just copies example_1 from the examples to generate a simple equilibrium.

In [1]:
# create and solve basic equilibrium.
# taking Example_1 and putting it all here to use as eq for testing probes

%load_ext autoreload
%autoreload 2 


import os
import numpy as np 

os.environ["ACTIVE_COILS_PATH"] = "../machine_configs/MAST-U/active_coils_from_UDA.pickle"
os.environ["PASSIVE_COILS_PATH"] = "../machine_configs/MAST-U/passive_coils.pickle"
os.environ["WALL_PATH"] = "../machine_configs/MAST-U/wall.pickle"
os.environ["LIMITER_PATH"] = "../machine_configs/MAST-U/limiter.pickle"
# the following environment variable is optional, 
# it contains data on an equilibrium that is representative for the tokamak 
# for more info please refer to Example_0
os.environ["EQUILIBRIUM_PATH"] = "../machine_configs/MAST-U/simple_initilization_equilibrium.pickle"
os.environ["PROBE_PATH"] = "../machine_configs/MAST-U/magnetic_probes.pickle"
# Now the machine can actually be built:
from freegsnke import build_machine
tokamak = build_machine.tokamak()

from freegsnke import equilibrium_update
from freegsnke.jtor_update import ConstrainPaxisIp
from freegsnke import GSstaticsolver


# seems to need to add in import freegs?
import freegs

In [2]:
# create equilibirum object
eq = equilibrium_update.Equilibrium(
    tokamak=tokamak,
    Rmin=0.1, Rmax=2.0,   # Radial range
    Zmin=-2.2, Zmax=2.2,  # Vertical range
    nx=65,   # Number of grid points in the radial direction
    ny=129,  # Number of grid points in the vertical direction
    # note that both nx and ny have to be of the form 2**i + 1 with i being an integer
    # psi=plasma_psi
)  


# create profile object
profiles = ConstrainPaxisIp(
    eq=eq,
    limiter=tokamak.limiter,
    paxis=8e3,
    Ip=6e5,
    fvac=0.5,
    alpha_m=1.8,
    alpha_n=1.2
)


# load nonlinear solver
GSStaticSolver = GSstaticsolver.NKGSsolver(eq)

# setup initial conditions....
# Set X-point coordinates
Rx = 0.6
Zx = 1.1
xpoints = [(Rx, -Zx),   
           (Rx,  Zx)]

# Set any desired isoflux constraints,
# these have structure (R1, Z1, R2, Z2)
# where (R1, Z1) and (R2, Z2) are 
# desired to be on the same flux contour.
Rmid = 1.4   # Outboard midplane
Rin = 0.35    # Inboard midplane
isoflux = [(Rx,Zx, Rx,-Zx), # link X-points
           (Rmid, 0, Rin, 0.0), # link inner and outer midplane points
           (Rmid, 0, Rx, Zx), # link outer midplane point and X-point

        #   some additional constraints
           (Rmid,0, 1.2,.7),
           (Rmid,0, 1.2,-.7),
           (Rx, Zx, .85, 1.7),
           (Rx, Zx, .75, 1.6),
           (Rx, Zx, Rin, 0.2),
           (Rx, Zx, Rin, 0.1),
           (Rx,-Zx, Rin, -0.1),
           (Rx,-Zx, Rin, -0.2),
           (Rx,-Zx, .85, -1.7),
           (Rx,-Zx, .75, -1.6),
           (Rx,-Zx, 0.45, -1.8),
           (Rx, Zx, 0.45,  1.8),
           ]
           
# Instantiate the constrain object
constrain = freegs.control.constrain(xpoints=xpoints, 
                                     isoflux=isoflux,
                                     gamma=5e-7
                                    )

# apply solver
# GSStaticSolver.solve(eq=eq, 
#                      profiles=profiles, 
#                      constrain=constrain, 
#                      target_relative_tolerance=1e-3)



Equilibrium initialised using file provided as part of the machine description.


In [3]:
#
# inverse_current_values = eq.tokamak.getCurrents()

# save coil currents to file
import pickle
with open('simple_diverted_currents_PaxisIp.pk', 'rb') as f:
    currents = pickle.load(file=f)

This is only the inverse solve, but will have coil and plasma currents that can be used by the probes.

Let's set this up now.

FORWARD SOLVING INSTEAD

In [4]:

for key in currents.keys():
    eq.tokamak[key].current = currents[key]

In [5]:
GSStaticSolver.solve(eq, 
                     profiles=profiles, 
                     constrain=None, 
                     target_relative_tolerance=1e-9)

## Testing probe

In [112]:
from freegsnke import magnetic_probes

# initialise a probe object.
# testing flux loops
test_probe = magnetic_probes.Probe(eq)
print("number of flux loops",len(test_probe.floops))
print("number of pickups",len(test_probe.pickups))



number of flux loops 102
number of pickups 398


Testing flux loop functions
- check init works apropriateluy 
- check output of functions gives correct objects (i.e. dimension of arrays is right)



In [27]:
# test initialisation of attributes
# print(test_probe.coil_names)
# print(test_probe.floop_pos)
# print(test_probe.floop_order)
# print(test_probe.number_floops)
# print(np.shape(test_probe.greens_psi_coils_floops))
# print(np.shape(test_probe.greens_psi_plasma_floops))

# testing methods
print(test_probe.get_coil_currents(eq).shape)
print(test_probe.get_plasma_current(eq).shape)
print(np.shape(test_probe.greens_psi_single_coil('Solenoid')))
print(np.shape(test_probe.psi_floop_all_coils(eq)))
print(np.shape(test_probe.green_psi_plasma(eq)))
print(np.shape(test_probe.psi_from_plasma(eq)))
print("-----")
print(test_probe.calculate_fluxloop_value(eq))
print(np.shape(test_probe.calculate_fluxloop_value(eq)))

(699,)
(65, 129)
(102,)
(102,)
(65, 129, 102)
(102,)
-----
[288.70987167 277.82199175 260.17917749 245.74684958 241.16261371
 240.31592486 231.56960942 288.70889577 277.82100645 260.17824827
 245.74603513 241.16172348 240.31503384 231.56925921 220.25552376
 220.25560825 259.75256109 234.45466599 207.80610333 185.45863124
 259.75145304 234.45326485 207.80454229 185.4579572  185.97568454
 185.92279189 185.91430357 185.95025312 185.95024989 185.91429924
 185.92278631 185.97567712 301.18114612 306.66228585 293.59361974
 278.92018275 260.00903932 253.58879618 205.95568402 301.39942281
 306.59830264 293.18794958 278.44882507 259.50462561 253.53669471
 205.91782994 294.86357459 294.84740449 272.60825034 272.58860103
 272.58776    294.84662614 272.60745073 294.8629124  307.43922716
 296.83773231 307.43295631 296.82308409 307.43376723 296.82398478
 307.43998718 296.83857103 280.82886431 226.36690315 280.82710238
 226.3908038  280.82866467 226.3926849  280.83053924 226.36859387
 176.36724021 124

In [55]:
# testing pickups 
# init attributes testing
print(test_probe.pickup_pos.shape)
# print(test_probe.pickup_order.shape)
print(test_probe.pickup_or.shape)
print(test_probe.number_pickups)
print(test_probe.greens_br_coils_pickup)
print(test_probe.greens_br_plasma_pickup.shape)

(398, 3)
(398, 3)
398
[]
(65, 129, 398)


Testing methods 


In [115]:
# print(test_probe.greens_BrBz_single_coil('Solenoid')[0])
# print(test_probe.greens_psi_all_coils().shape)
# print(test_probe.greens_BrBz_all_coils()[0].shape)
# print(test_probe.greens_BrBz_plasma(eq))
print(test_probe.greens_br_plasma_pickup.shape)
print(test_probe.Br(eq).shape)
print(test_probe.Btor(eq).shape)
print(test_probe.calculate_pickup_value(eq).shape)
print(test_probe.pickup_or.shape)
print(test_probe.Br(eq).shape)

(65, 129, 398)
(398,)
(398,)
(398,)
(398, 3)
(398,)


In [56]:
print(test_probe.greens_psi_all_coils())

[[9.23550579e-06 9.52072605e-06 9.87696149e-06 ... 1.48585486e-05
  1.35480022e-05 1.35307382e-05]
 [5.39443681e-06 5.24454845e-06 4.91713763e-06 ... 1.38849227e-05
  1.55982871e-05 1.55721500e-05]
 [1.16102119e-05 1.05546269e-05 9.15259807e-06 ... 2.88062688e-06
  6.32228239e-06 6.36481900e-06]
 ...
 [6.02304994e-07 7.04983631e-07 9.06427670e-07 ... 3.67336423e-08
  3.08944907e-08 3.08518888e-08]
 [5.46081779e-07 6.40916236e-07 8.26836715e-07 ... 3.84839117e-08
  3.16748985e-08 3.16259552e-08]
 [6.01870992e-07 7.07285661e-07 9.15051279e-07 ... 3.51062058e-08
  2.89017642e-08 2.88573175e-08]]


In [113]:
arr = np.array([['r1','th1','z1'],['r2','th2','z2']])
arr2 = arr.shape
arr.transpose()





array([['r1', 'r2'],
       ['th1', 'th2'],
       ['z1', 'z2']], dtype='<U3')